In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
import pandas as pd
import os, tqdm, warnings

warnings.filterwarnings('ignore')

2025-11-16 03:35:53.412015: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 03:35:54.366647: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 03:35:56.731912: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/inter/miniconda/envs/tf-gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning

In [3]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', cls_token='<usr>',
                                          pad_token='<pad>', sep_token='<sys>', unk_token='<unk>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

I0000 00:00:1763231767.033794    2619 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TF

In [16]:
df = pd.read_csv('/home/inter/chat_MBTI/make_dataset/dataset/dialog_chatbot.csv')

In [17]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(df.loc[df['Q'].str.contains('\*')].index, axis=0, inplace=True)
df.drop(df.loc[df['A'].str.contains('\*')].index, axis=0, inplace=True)
df.drop(df.loc[df['Q'].str.len() >= 50].index, axis=0, inplace=True)
df.drop(df.loc[df['A'].str.len() >= 50].index, axis=0, inplace=True)
df

,Q,Q_intent,A,A_intent
0,스페인 마드리드에 여행가면 하고 싶은거 있어?,(지시) 질문하기,나는 키키 축구 보고 싶네,(단언) 주장하기
1,나는 키키 축구 보고 싶네,(단언) 주장하기,축구팀 어디 경기 말이야?,(지시) 질문하기
2,축구팀 어디 경기 말이야?,(지시) 질문하기,거기 레알 마드리드 홈구장이 있잖아 하하,(단언) 진술하기
3,거기 레알 마드리드 홈구장이 있잖아 하하,(단언) 진술하기,아 그 베르나베우였나?,(지시) 질문하기
4,아 그 베르나베우였나?,(지시) 질문하기,응 키키 거기서 홈 경기를 한번 보고 싶네,(단언) 주장하기
...,...,...,...,...
731723,리뷰 보면 그런 말 많이 나오더라.,(단언) 진술하기,"헐, 엄청 긴데 피곤했겠다.",(단언) 주장하기
731724,그걸 하루 만에 보다니 엄청 재밌었나 보네.,(단언) 주장하기,오징어 게임 시즌 2도 나올까?,(지시) 질문하기
731725,피곤한 게 잊을 만큼 재밌었거든.,(단언) 주장하기,오징어 게임 시즌 2도 나올까?,(지시) 질문하기
731726,피곤한 게 잊을 만큼 재밌었거든.,(단언) 주장하기,나는 시즌 1부터 봐야겠어.,(언약) 약속하기(제3자와)/(개인적 수준)


In [18]:
intet_label = list(df['Q_intent'].unique())

label_dict = {}

for idx, intent_lab in enumerate(intet_label) :
    label_dict[idx] = intent_lab

label_dict

{0: '(지시) 질문하기',
 1: '(단언) 주장하기',
 2: '(단언) 진술하기',
 3: '(지시) 충고/제안하기',
 4: '(표현) 부정감정 표현하기',
 5: '(언약) 약속하기(제3자와)/(개인적 수준)',
 6: '(표현) 감사하기',
 7: '턴토크 사인(관습적 반응)',
 8: '(지시) 명령/요구하기',
 9: '(지시) 부탁하기',
 10: '(표현) 긍정감정 표현하기',
 11: '(단언) 반박하기',
 12: '(표현) 인사하기',
 13: '(표현) 사과하기',
 14: '(언약) 거절하기',
 15: '(언약) 위협하기',
 16: '(선언/위임하기)'}

In [19]:
Q_int_li = list(set(df['Q_intent'].unique()))
A_int_li = list(set(df['A_intent'].unique()))

for i in Q_int_li :
  print(f"{i} / Q : {df.loc[df['Q_intent'] == i, 'Q_intent'].count()} / A : {df.loc[df['A_intent'] == i, 'A_intent'].count()}")

(단언) 주장하기 / Q : 196668 / A : 237055
(표현) 인사하기 / Q : 246 / A : 468
(지시) 질문하기 / Q : 217882 / A : 169834
(표현) 긍정감정 표현하기 / Q : 4347 / A : 5910
(언약) 약속하기(제3자와)/(개인적 수준) / Q : 9672 / A : 12511
(표현) 감사하기 / Q : 942 / A : 1782
(지시) 명령/요구하기 / Q : 2612 / A : 2654
(언약) 거절하기 / Q : 275 / A : 424
(단언) 진술하기 / Q : 223375 / A : 216109
(선언/위임하기) / Q : 33 / A : 65
(지시) 충고/제안하기 / Q : 26589 / A : 29405
(언약) 위협하기 / Q : 32 / A : 38
(단언) 반박하기 / Q : 2296 / A : 3013
(표현) 부정감정 표현하기 / Q : 5260 / A : 5961
(표현) 사과하기 / Q : 346 / A : 715
턴토크 사인(관습적 반응) / Q : 3038 / A : 7625
(지시) 부탁하기 / Q : 3849 / A : 3893


In [20]:
train_data = pd.DataFrame()

for v,k in label_dict.items():
    len_ = df.loc[df['Q_intent'] == k,'Q_intent'].count()
    if len_ > 10000 :
        train_data = pd.concat([train_data,df.loc[df['Q_intent'] == k].sample(n=10000)],ignore_index=True)
    else :
        train_data = pd.concat([train_data,df.loc[df['Q_intent'] == k]],ignore_index=True)

for v,k in label_dict.items():
    len_ = df.loc[df['A_intent'] == k,'A_intent'].count()
    if len_ > 10000 :
        train_data = pd.concat([train_data,df.loc[df['A_intent'] == k].sample(n=10000)],ignore_index=True)
    else :
        train_data = pd.concat([train_data,df.loc[df['A_intent'] == k]],ignore_index=True)

train_data.drop(train_data[train_data.duplicated()].index, axis=0, inplace=True)
train_data = train_data.sample(frac=1).reset_index(drop=True)
train_data

,Q,Q_intent,A,A_intent
0,사랑은 순수해야 하는거 아니야?,(지시) 질문하기,그 어떠한 순수함도 현실 앞에서는 어림없는 법~,(단언) 주장하기
1,성형은 하기 싫어.,(단언) 주장하기,그럼 마사지 같은 거 해.,(지시) 충고/제안하기
2,오빠 이번에 올림픽 한 거 봤어?,(지시) 질문하기,응 나 축구랑 배구는 봤어 키키,(단언) 진술하기
3,맞아 화면이 커서 좋아,(표현) 긍정감정 표현하기,영화관이 소리도 웅장하고,(단언) 주장하기
4,그래 귀엽잖아 꼬리만 해도 너무 이쁠 거 같아,(단언) 주장하기,분명 좋은 애견미용실 만날 거야,(지시) 충고/제안하기
...,...,...,...,...
145462,설탕 빠지면 섭하지,(단언) 주장하기,맞아 키키,턴토크 사인(관습적 반응)
145463,잡화점 그 책은 20%정도 할인 하던데요?,(단언) 진술하기,좋다 엄마 읽게 하나 사주렴,(지시) 부탁하기
145464,정말 감사드려서 어쩔 줄 모르겠어ㅠㅠ,(표현) 감사하기,아무런 조건 없이 가불 해주셨으니 얼마나 감사한 마음이 들겠어...,(단언) 주장하기
145465,주택쪽이 성과가 좋으니?,(지시) 질문하기,제발 나왔으면 좋겠다,(표현) 긍정감정 표현하기


In [27]:
batch_size = 8

In [28]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
    yield bos_token + sent + eos_token

In [29]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [30]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [31]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[    0     2 46260 14866  9685  9277  6853  9320  7991   406     4  9022
  13611 14866  8707  7235 10440  9497  9180  9105  7482 13278  9236   468
      1     3     3]
 [    0     2  9096 15441 11487 16238   389     4 18519  9109 13921  9239
   9122  9098   389     1     3     3     3     3     3     3     3     3
      3     3     3]
 [    0     2 28737  9018 18554 14604  9036  9122   739  7662  8006   406
      4 10223  9063 18309  7380  9208 11519   739  7662  8006 10252  8511
      1     3     3]
 [    0     2 15247  9192 11383 27138 12011     4 10584 10211 12867  7235
  48829     1     3     3     3     3     3     3     3     3     3     3
      3     3     3]
 [    0     2 10723  9504  8038  8162  7965 22062  7489 13927 12371  9018
   7750  9122 27511     4 11296 10586  9831  6877  7584  8091  7892 25891
   9122  7991     1]
 [    0     2 25689  9162  6949  9443 15234 23775  9098 11213 13358 12016
  10652     4  9044 17469 23869  8346  7055 11560  7661  7991  6872  9

In [32]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [33]:
if len(train_data) % batch_size == 0:
    steps = len(train_data) // batch_size
else :
    steps = len(train_data) // batch_size + 1

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

In [12]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  print(sentence)
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [13]:
def chat():
    while 1:
        sen = input("user > ").strip()
        if sen == '끝':
            break
        respon = return_answer_by_chatbot(sen)
        print("Chatbot > {}".format(respon.strip()))

In [ ]:
MODEL_SAVE_PATH = os.path.join("koGPT_Chatbot") # change this to your preferred location
print(MODEL_SAVE_PATH)
print('='*50)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")

else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)